In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
positiveData = []
with open('/positiveData.txt') as file:

  for i in file:
    positiveData.append(i.strip())

In [ ]:
positiveData

['မြန်မာကားကြည့်ရတာကြိုက်တယ်', 'ဒီသရဲကားကောင်းတယ်', 'အိမ်စာလုပ်ရတာကြိုက်တယ်']

In [ ]:
negativeData = []
with open('/negativeData.txt') as file:

  for i in file:
    negativeData.append(i.strip())

In [ ]:
negativeData

['မြန်မာကားကြည့်ရတာမကြိုက်ဘူး', 'ဒီသရဲကားမကောင်းဘူး', 'အိမ်စာမလုပ်ချင်ဘူး']

In [ ]:
import re #regression
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pickle

## Data Preprocessing

In [ ]:
positive_df = pd.DataFrame()
positive_df['message'] = positiveData
positive_df['label'] = [1]*len(positiveData)
print(positive_df)

                      message  label
0  မြန်မာကားကြည့်ရတာကြိုက်တယ်      1
1           ဒီသရဲကားကောင်းတယ်      1
2      အိမ်စာလုပ်ရတာကြိုက်တယ်      1


In [ ]:
negative_df = pd.DataFrame()
negative_df['message'] = negativeData
negative_df['label'] = [0]*len(negativeData)
print(negative_df)

                       message  label
0  မြန်မာကားကြည့်ရတာမကြိုက်ဘူး      0
1           ဒီသရဲကားမကောင်းဘူး      0
2           အိမ်စာမလုပ်ချင်ဘူး      0


In [ ]:
train_positive_df = int(positive_df.shape[0]*0.7)
test_positive_df = positive_df.shape[0] - train_positive_df

positive_df = positive_df.sample(frac=1)
train_positive = positive_df.head(train_positive_df)
print(train_positive)
test_positive = positive_df.tail(test_positive_df)
print(test_positive)

                      message  label
1           ဒီသရဲကားကောင်းတယ်      1
0  မြန်မာကားကြည့်ရတာကြိုက်တယ်      1
                  message  label
2  အိမ်စာလုပ်ရတာကြိုက်တယ်      1


In [ ]:
train_negative_df = int(negative_df.shape[0]*0.7)
test_negative_df = negative_df.shape[0] - train_negative_df

negative_df = negative_df.sample(frac=1)
train_negative = negative_df.head(train_negative_df)
print(train_negative)
test_negative = negative_df.tail(test_negative_df)
print(test_negative)

                       message  label
1           ဒီသရဲကားမကောင်းဘူး      0
0  မြန်မာကားကြည့်ရတာမကြိုက်ဘူး      0
              message  label
2  အိမ်စာမလုပ်ချင်ဘူး      0


In [ ]:
train  = pd.concat([train_positive, train_negative], axis = 0)
test  = pd.concat([test_positive, test_negative], axis = 0)

In [ ]:
train

,message,label
1,ဒီသရဲကားကောင်းတယ်,1
0,မြန်မာကားကြည့်ရတာကြိုက်တယ်,1
1,ဒီသရဲကားမကောင်းဘူး,0
0,မြန်မာကားကြည့်ရတာမကြိုက်ဘူး,0


In [ ]:
test

,message,label
2,အိမ်စာလုပ်ရတာကြိုက်တယ်,1
2,အိမ်စာမလုပ်ချင်ဘူး,0


In [ ]:
train['message']

1              ဒီသရဲကားကောင်းတယ်
0     မြန်မာကားကြည့်ရတာကြိုက်တယ်
1             ဒီသရဲကားမကောင်းဘူး
0    မြန်မာကားကြည့်ရတာမကြိုက်ဘူး
Name: message, dtype: object

## Tokenization and Padding

In [ ]:
vocab_size = 15

tokenizer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
tokenizer.adapt(train['message'])
sequences = tokenizer(train['message'])
padded = pad_sequences(sequences)

In [ ]:
tokenizer

In [ ]:
sequences

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[5],
       [3],
       [4],
       [2]])>

In [ ]:
padded

array([[5],
       [3],
       [4],
       [2]], dtype=int32)

## Modeling

In [ ]:
model = Sequential([
    Embedding(vocab_size, 64),
    LSTM(16),
    Dense(500, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(1, activation='sigmoid')
])
print(model)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 100
history = model.fit(padded, np.array(train['label']), epochs=num_epochs)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.6931 - accuracy: 0.2500
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6904 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6878 - accuracy: 0.7500
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6843 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6799 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6740 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6664 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 21ms/step - loss: 0.6568 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6448 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6302 - accuracy: 1.0000
Epoch 11/100
1/1 [===

In [ ]:
model.save('sentiment_analysis_burmese')

## Evaluation

In [ ]:
loaded_model = tf.keras.models.load_model('sentiment_analysis_burmese')
test_sequence = tokenizer(test['message'])
padded_test_sequence =pad_sequences(test_sequence)

#predict sentiment
prediction = loaded_model.predict(padded_test_sequence)

y_pred = []
threshold = 0.5
for  i in prediction:
  current_val = i[0]
  if (current_val>=threshold):
    y_pred.append(1)
  else:
    y_pred.append(0)


1/1 [==============================] - 0s 406ms/step


In [ ]:
# မြန်မာကားကြည့်ရတာကြိုက်တယ် , ဒီဟာကတော်တော်မိုက်တယ်နော် , ဒီသရဲကားမကောင်းဘူး , အိမ်စာမလုပ်ချင်ဘူး , အိမ်စာလုပ်ရတာကြိုက်တယ်
inputs = ['ဒီသရဲကားမကောင်းဘူး']
print(type(inputs))
print(f"Test String: {inputs}")

testing_sequences = tokenizer(inputs)
testing_padded = pad_sequences(testing_sequences)
ans = model.predict(testing_padded)
print(f"Confidence:{ans[0][0]}")

#convert prediction to sentiment label
sentiment_label = 'positive' if ans > 0.5 else 'negative'

print(f"Test String: {inputs}")
print(f"Predicted Sentiment: {sentiment_label}")

<class 'list'>
Test String: ['ဒီသရဲကားမကောင်းဘူး']
1/1 [==============================] - 0s 374ms/step
Confidence:9.29130783333676e-06
Test String: ['ဒီသရဲကားမကောင်းဘူး']
Predicted Sentiment: negative


In [ ]:
file_name = 'sentiment_analysis_burmese.sav'
pickle.dump(loaded_model,open(file_name,'wb'))